# Load the dataset

In [64]:
with open("./datasets/football_key.tsv") as f:
    # Each line is of form: <country_id> <country_name>
    def fmt(line):
        return (int(line[0])-1, line[1].strip('"'))
    data_key = [fmt(line.strip().split()) for line in f if line[0] != '*']

In [65]:
with open("./datasets/football_pairs.tsv") as f:
    # Each line is of form: <country_a_id> <country_b_id> <number_of_players>
    def fmt(pair):
        return (int(pair[0])-1, int(pair[1])-1, 1)
    data_pairs = [fmt(line.strip().split()) for line in f if line[0] != '*']

Create the neighbours[] sets

In [66]:
neighbours = [set() for _ in range(len(data_key))]
for p in data_pairs:
    neighbours[p[0]].add(p[1])
    neighbours[p[1]].add(p[0])

Define the similarity metric: graph distance.

Method to compute graph distance without Dijkstra or similar taken from _Scalable Proximity Estimation and Link Prediction in Online Social Networks_ - Han Hee Song, Tae Won Cho, Vacha Dave, Yin Zhang, Lili Qiu:

We initialize S = {x} and D = {y}. In each step we either expand set S to include its members’ neighbors (i.e., S = S ∪ {v|⟨u, v⟩ ∈ E ∧ u ∈ S}) or expand set D to include its members’ inverse neighbors (i.e., D = D ∪ {u|⟨u, v⟩ ∈ E ∧ v ∈ D}). We stop whenever S ∩ D != ∅ . The number of steps taken so far gives the shortest path distance. For efficiency, we always expand the smaller set between S and D in each step.

In [67]:
def similarity_GD(x, y, ignore_set=None):
    MAX_DIST = 6
    def expand(nset):
        for n in set(nset):
            for m in neighbours[n]:
                if (ignore_set is not None and
                   ((n, m) in ignore_set or (m, n) in ignore_set)):
                    # We should calculate without this link,
                    # as it is in the test set for this iter.
                    continue
                nset.add(m)

    s = set([x])
    d = set([y])
    dist = 0
    while len(s & d) == 0 and dist <= MAX_DIST:
        dist += 1
        if len(d) < len(s):
            expand(d)
        else:
            expand(s)
    return -dist

Compute the similarities across the dataset.

In [68]:
def compute_similarities(ignore_set=None):
    # S_GD[x][y] contains the similarity of nodes x and y using the Graph Distance (GD) metric.
    S_GD = [[0 for _ in range(len(data_key))] for _ in range(len(data_key))]
    for i in range(len(data_key)-1):
        for j in range(0, len(data_key)):
            S_GD[i][j] = similarity_GD(i, j, ignore_set=ignore_set)
    return S_GD


In [69]:
# A quick eyeball check of a subset of the data.

# Similarities with all links included.
S_GD = compute_similarities()

num_to_print = len(data_key)//2
print(' '*4 + ' '.join(d[1] for d in data_key[:num_to_print]))
print('\n'.join(data_key[i][1] + ' ' + ','.join('{:>3}'.format(c) for c in S_GD[i][:num_to_print]) for i in range(num_to_print)))

    ARG AUT BEL BGR BRA CHE CHL CMR COL DEU DNK ESP FRA GBR GRE HRV IRN
ARG   0, -2, -2, -2, -2, -3, -1, -2, -1, -2, -2, -1, -2, -2, -3, -2, -3
AUT  -2,  0, -2, -2, -2, -2, -2, -1, -2, -1, -2, -1, -1, -1, -2, -1, -2
BEL  -2, -2,  0, -2, -2, -2, -2, -2, -2, -1, -2, -2, -1, -2, -2, -2, -2
BGR  -2, -2, -2,  0, -2, -3, -3, -2, -2, -1, -2, -1, -2, -3, -3, -2, -2
BRA  -2, -2, -2, -2,  0, -3, -2, -2, -1, -2, -2, -1, -1, -2, -3, -2, -3
CHE  -3, -2, -2, -3, -3,  0, -3, -3, -3, -2, -3, -2, -2, -2, -4, -3, -3
CHL  -1, -2, -2, -3, -2, -3,  0, -2, -2, -2, -2, -2, -2, -2, -3, -2, -3
CMR  -2, -1, -2, -2, -2, -3, -2,  0, -2, -1, -2, -1, -1, -2, -1, -2, -2
COL  -1, -2, -2, -2, -1, -3, -2, -2,  0, -2, -2, -1, -2, -2, -3, -2, -3
DEU  -2, -1, -1, -1, -2, -2, -2, -1, -2,  0, -1, -1, -1, -2, -2, -1, -1
DNK  -2, -2, -2, -2, -2, -3, -2, -2, -2, -1,  0, -1, -2, -1, -3, -2, -2
ESP  -1, -1, -2, -1, -1, -2, -2, -1, -1, -1, -1,  0, -2, -2, -2, -1, -2
FRA  -2, -1, -1, -2, -1, -2, -2, -1, -2, -1, -2, -2,  0, -2, -2,

Split the data into 10 disjoint subsets.

In [70]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for it in range(0, len(l), n):
        yield l[it:it + n]
        
e = []
predict = []
for i in range(len(data_key)):
    for j in range(i+1, len(data_key)):
        if i in neighbours[j]:
            e.append((i, j))
        else:
            predict.append((i, j))
            
# e now contains all link pairs
# predict contains all non-existing links from the original data
# each pair is a tuple (a, b), where a < b

# We now randomly shuffle this list
import random
random.shuffle(e)

print('len(e)', len(e))
print('len(e)//10 =', len(e)//10)

# Create e_prime, a list of 10 partitions
e_prime = (list(chunks(e, len(e)//10 + 1)))

# TODO(iandioch): Figure out why the following line is necessary?
# e_prime = e_prime[0]

# The following is a quick eyeball test to make sure the partitions look ok.
print('10 subsets:')
for i in range(len(e_prime)):
    entry = e_prime[i]
    print(entry)


len(e) 118
len(e)//10 = 11
10 subsets:
[(24, 28), (11, 21), (24, 32), (12, 21), (1, 9), (17, 23), (0, 11), (0, 8), (1, 7), (7, 26), (4, 11), (5, 34)]
[(13, 24), (13, 25), (10, 24), (12, 33), (13, 29), (12, 34), (13, 15), (3, 31), (13, 17), (7, 31), (11, 25), (4, 8)]
[(17, 24), (9, 11), (9, 15), (12, 23), (5, 23), (12, 30), (9, 25), (9, 28), (10, 17), (9, 23), (7, 17), (1, 13)]
[(11, 23), (7, 12), (31, 34), (2, 23), (23, 34), (3, 9), (11, 27), (10, 29), (12, 17), (9, 24), (9, 33), (19, 33)]
[(17, 34), (9, 32), (3, 26), (1, 17), (11, 17), (23, 24), (13, 28), (7, 11), (23, 32), (17, 25), (21, 30), (15, 31)]
[(11, 15), (1, 15), (22, 27), (14, 28), (11, 28), (4, 27), (17, 33), (10, 31), (9, 17), (1, 34), (4, 26), (13, 34)]
[(7, 19), (12, 20), (11, 33), (0, 6), (9, 30), (2, 12), (6, 17), (13, 18), (24, 34), (7, 9), (7, 14), (2, 17)]
[(19, 20), (9, 16), (4, 19), (17, 21), (9, 10), (13, 32), (9, 34), (0, 17), (12, 29), (8, 32), (2, 24), (15, 17)]
[(3, 11), (10, 11), (23, 31), (13, 33), (8, 11)

In [76]:
aucs = []
n1s = []
n2s = []
n3s = []
ns = []

# Column headings.
print('\t\tn1   \tn2   \tn3   \tAUC')

# Iterate across the 10 folds.
for i in range(10):
    test = e_prime[i]
    S_CN = compute_similarities(ignore_set=test)
    
    n1 = 0 # missing_link > nonexistant_link
    n2 = 0 # missing_link = nonexistant_link
    n3 = 0 # missing_link < nonexistant_link
    n = 0 # total link comparisons
    for missing_link in test:
        a_score = S_CN[missing_link[0]][missing_link[1]]
        for nonexistant_link in predict:
            b_score = S_CN[nonexistant_link[0]][nonexistant_link[1]]
            if abs(a_score-b_score) < 0.0005:
                n2 += 1
            elif a_score > b_score:
                n1 += 1
            else:
                n3 += 1
            n += 1
    auc = (n1 + 0.5*n2)/(n)
    aucs.append(auc)
    n1s.append(n1)
    n2s.append(n2)
    n3s.append(n3)
    ns.append(n)
    print('Fold {:<2}:\t{:<5}\t{:<5}\t{:<5}\t{:<.6f}'.format(i+1, n1, n2, n3, auc))

def avg(seq):
    return sum(seq)/len(seq)

print('Average:\t{:<5}\t{:<5}\t{:<5}\t{:<.6f}'.format(int(round(avg(n1s))), int(round(avg(n2s))), int(round(avg(n3s))), avg(aucs)))

		n1   	n2   	n3   	AUC
Fold 1 :	2072 	3361 	291  	0.655573
Fold 2 :	1794 	3069 	861  	0.581499
Fold 3 :	2280 	3444 	0    	0.699161
Fold 4 :	2086 	3084 	554  	0.633823
Fold 5 :	1725 	3120 	879  	0.573899
Fold 6 :	1636 	2777 	1311 	0.528389
Fold 7 :	1745 	2275 	1704 	0.503581
Fold 8 :	1674 	2522 	1528 	0.512753
Fold 9 :	1803 	3063 	858  	0.582547
Fold 10:	1496 	2682 	592  	0.594759
Average:	1831 	2940 	858  	0.586599
